# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Simón Botero Arango

**ID:** 0004467036


Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [50]:
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [51]:
id_upb = "000446036"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 469 to 68987
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  string
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), string(1)
memory usage: 1.1 MB


Ejemplo de algunas filas del dataset

In [52]:
data_reviews.sample(5)

,author_review_desc,author_rating
32786,\nEsta secuela me encanto y es una de las mejo...,10
26099,\n“Atrapado en el tiempo” constituye un filme ...,7
140587,\nNuestra candidata a mejor película de animac...,6
15879,\nAmo la psicología y me fascinan las teorías ...,1
92156,"\nY eso que decía, década tras década, que dej...",9


Evaluar los valores nulos

In [53]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [54]:
print(f"cantidad de duplicados: {data_reviews.duplicated().sum()}")
data_reviews = data_reviews.drop_duplicates()

cantidad de duplicados: 2


No hay  valores duplicados porque fueron eliminados

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [55]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [56]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [58]:
import nltk

nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
import numpy as np

metricas_texto = pd.DataFrame(
    {
        "longitud_caracteres": X_train.str.len(),
        "longitud_palabras": X_train.str.split().str.len(),
        "proporcion_mayusculas": X_train.str.count(r"[A-ZÁÉÍÓÚÜÑ]").div(
            X_train.str.len().replace({0: np.nan})
        ),
    }
)

metricas_texto.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).T

,count,mean,std,min,10%,25%,50%,75%,90%,max
longitud_caracteres,39998.0,1388.545777,1065.601333,29.0,404.0,625.0,1078.0,1831.0,2770.0,6062.0
longitud_palabras,39998.0,236.732837,180.394684,3.0,69.0,107.0,185.0,313.0,471.0,1106.0
proporcion_mayusculas,39998.0,0.016887,0.011921,0.0,0.007937,0.010905,0.014856,0.02,0.026586,0.724638


In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words("spanish")
vectorizer_unigram = CountVectorizer(stop_words=spanish_stopwords, max_features=30)
frecuencias_unigram = pd.Series(
    vectorizer_unigram.fit_transform(X_train).sum(axis=0).A1,
    index=vectorizer_unigram.get_feature_names_out(),
).sort_values(ascending=False)

vectorizer_bigram = CountVectorizer(
    stop_words=spanish_stopwords, ngram_range=(2, 2), max_features=30
)
frecuencias_bigram = pd.Series(
    vectorizer_bigram.fit_transform(X_train).sum(axis=0).A1,
    index=vectorizer_bigram.get_feature_names_out(),
).sort_values(ascending=False)

display(frecuencias_unigram.head(15).to_frame("frecuencia"))
display(frecuencias_bigram.head(15).to_frame("frecuencia"))

,frecuencia
película,66741
si,28320
historia,25081
bien,21912
ser,21266
cine,19463
ver,16609
tan,16283
personajes,16202
mejor,15931


,frecuencia
banda sonora,3296
efectos especiales,2446
obra maestra,2176
si bien,1719
ciencia ficción,1706
mejor película,1688
ver película,1633
cada vez,1526
puede ser,1514
tal vez,1495


## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [61]:
import re

ACCENT_TRANSLATION = str.maketrans(
    {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
        "à": "a",
        "è": "e",
        "ì": "i",
        "ò": "o",
        "ù": "u",
        "ä": "a",
        "ë": "e",
        "ï": "i",
        "ö": "o",
    }
)

BRACKETS_RE = re.compile(r"\[[^\[\]]*\]")
URL_RE = re.compile(r"https?://\S+|www\.\S+")
HTML_RE = re.compile(r"<[^>]+>")
NON_ASCII_RE = re.compile(r"[^\x00-\x7Fñ ]")


def clean_spanish_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.translate(ACCENT_TRANSLATION)
    text = BRACKETS_RE.sub(" ", text)
    text = URL_RE.sub(" ", text)
    text = HTML_RE.sub(" ", text)
    text = text.replace("\n", " ")
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"[^\w\sñ]", " ", text)
    text = NON_ASCII_RE.sub(" ", text)
    text = text.replace("_", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [62]:
X_train = X_train.apply(clean_spanish_text)
X_test = X_test.apply(clean_spanish_text)

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [63]:
from nltk.stem.snowball import SnowballStemmer


def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in spanish_stopwords]
    return " ".join(filtered_words)


X_train = X_train.apply(remove_stopwords)
X_test = X_test.apply(remove_stopwords)


# Crear el stemmer para español
stemmer = SnowballStemmer("spanish")


# Función para aplicar stemming
def apply_stemming(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)


# Aplicarlo después de los otros pasos de limpieza
X_train = X_train.apply(apply_stemming)
X_test = X_test.apply(apply_stemming)


def handle_negation(text):
    # Reemplazar 'no ' con 'no-' para unirlo a la siguiente palabra
    return text.replace("no ", "no-")


X_train = X_train.apply(handle_negation)


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(max_features=5000)
X_train = vectorizer_tfidf.fit_transform(X_train)
X_test = vectorizer_tfidf.transform(X_test)


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [65]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1.0, solver="liblinear", random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [66]:
y_pred = model.predict(X_test)


### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [67]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.8083
Precision: 0.8064746914684314
Recall: 0.84375
F1 Score: 0.8246913580246914
ROC AUC Score: 0.8056808419243986
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79      4656
           1       0.81      0.84      0.82      5344

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



**Guardar el modelo**

In [68]:
from pathlib import Path
import joblib

models_dir = Path("models")
models_dir.mkdir(parents=True, exist_ok=True)

artifact_path = models_dir / "logreg_sentiment.joblib"
joblib.dump({"vectorizer": vectorizer_tfidf, "model": model}, artifact_path)
print(f"Modelo guardado en {artifact_path}")

Modelo guardado en models/logreg_sentiment.joblib


## Librerías Usadas

In [69]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.12.11
IPython version      : 9.6.0

pandas   : 2.3.3
watermark: 2.5.0
joblib   : 1.5.2
numpy    : 2.3.4
sklearn  : 1.7.2
re       : 2.2.1
nltk     : 3.9.2



---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)